**No tratamento dos dados, removi as colunas que continham uma alta proporção de valores nulos e zeros simultaneamente, estabelecendo um limite de 40%. Em seguida, imputei os valores ausentes utilizando a mediana. Para lidar com o desbalanceamento de classes, utilizei o algoritmo RandomOverSampler(sampling_strategy=1). Posteriormente, normalizei os dados utilizando o método StandardScaler(). Utilizei os algoritmos VarianceThreshold(threshold=0.3), SmartCorrelatedSelection() e RFE() para reduzir a dimensionalidade dos dados, finalizando com 20 colunas selecionadas.**

# Importação das bibliotecas

In [23]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.feature_selection import RFE, VarianceThreshold
from feature_engine.selection import SmartCorrelatedSelection
from sklearn.ensemble import RandomForestClassifier
import warnings
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import pickle
import os
warnings.filterwarnings("ignore", category=FutureWarning, module="seaborn")
warnings.simplefilter(action="ignore", category=FutureWarning)

# Coleta dos dados

In [17]:
cal_and_teste= pd.read_csv("dados/air_system_present_year.csv")
treino = pd.read_csv("dados/air_system_previous_years.csv")

# Tratando os dados

In [25]:
treino_trat = treino.copy()
cal_and_teste_trat = cal_and_teste.copy()

## Eliminando colunas

In [19]:
# Tratamento dos dados 
## Remover colunas com valores ausentes e 0 maior de 40%
def remove_valores_ausentes_zero(treino, teste, label):
    lista_valores_ausentes_zero = []
    for coluna in treino.columns:
        if coluna == label:
            continue
        else:
            if (treino[coluna].isnull().sum() + sum(treino[coluna] == 0)) / len(
                treino
            ) >= 0.4:
                lista_valores_ausentes_zero.append(coluna)
    treino.drop(columns=lista_valores_ausentes_zero, inplace=True)
    teste.drop(columns=lista_valores_ausentes_zero, inplace=True)
    ## Selecionar linhas com pelo menos 50 colunas preenchidas
    treino = treino[treino.notnull().sum(axis=1) >= 50]
    teste = teste[teste.notnull().sum(axis=1) >= 50]
    return treino, teste


def colunas_numericas(treino, teste, label):
    for coluna in treino.columns:
        if coluna == label:
            continue
        else:
            treino[coluna] = pd.to_numeric(treino[coluna], errors="coerce")
            teste[coluna] = pd.to_numeric(teste[coluna], errors="coerce")
    return treino, teste

In [26]:
##Transformar colunas em numéricas
treino_trat, test_trat = colunas_numericas(treino_trat, cal_and_teste_trat, "class")
## Remover colunas com valores ausentes e 0 maior de 40% e Selecionar linhas com pelo menos 50 colunas preenchidas
treino_trat, cal_and_teste_trat = remove_valores_ausentes_zero(treino_trat, cal_and_teste_trat, "class")
##Transfomando as classes em numericas
treino_trat["class"] = np.where(treino_trat["class"] == "pos", 1, 0)
cal_and_teste_trat["class"] = np.where(cal_and_teste_trat["class"] == "pos", 1, 0)

## Divisao e y e x
y_treino = treino_trat["class"]
x_treino = treino_trat.drop(columns="class")
y_cal_and_teste = cal_and_teste_trat["class"]
x_cal_and_teste = cal_and_teste_trat.drop(columns="class")

/tmp/ipykernel_30913/3814298814.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cal_and_teste_trat["class"] = np.where(cal_and_teste_trat["class"] == "pos", 1, 0)


## Dividindo dados de teste em dados de calibracao e teste

In [27]:
x_calibracao, x_teste, y_calibracao, y_teste = train_test_split(x_cal_and_teste, y_cal_and_teste, test_size=0.5, random_state=42, stratify=y_cal_and_teste)

In [33]:
y_treino.value_counts()

class
0    58609
1      997
Name: count, dtype: int64

In [28]:



print("Shape do x_treino:", x_treino.shape)
print("Shape do y_treino:", y_treino.shape)
print("Shape do x_calibracao:", x_calibracao.shape )
print("Shape do y_calibracao",y_calibracao.shape )
print("Shape do x_teste:", x_teste.shape)
print("Shape do y_teste:", y_teste.shape)

Shape do x_treino: (59606, 99)
Shape do y_treino: (59606,)
Shape do x_calibracao: (7945, 99)
Shape do y_calibracao (7945,)
Shape do x_teste: (7946, 99)
Shape do y_teste: (7946,)


## Desbalanceando os dados

In [29]:
ros = RandomOverSampler(sampling_strategy=1,random_state=42)
x_treino_res, y_treino_res= ros.fit_resample(x_treino, y_treino)

In [30]:
x_treino_res.shape,y_treino_res.shape

((117218, 99), (117218,))

## Utlizando PIPELINE para aplicar SimpleImputer, StandardScaler, VarianceThreshold SmartCorrelatedSelection e RFE

In [34]:
pipe = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
        ("variance", VarianceThreshold(threshold=0.3)),
        (
            "correlacao",
            SmartCorrelatedSelection(selection_method="variance", threshold=0.7),
        ),
        ("rfe", RFE(estimator=RandomForestClassifier(), n_features_to_select=20)),
        #("clf", RandomForestClassifier()),
    ]
)

In [35]:
pipe.fit(x_treino_res, y_treino_res)

Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler()),
                ('variance', VarianceThreshold(threshold=0.3)),
                ('correlacao',
                 SmartCorrelatedSelection(selection_method='variance',
                                          threshold=0.7)),
                ('rfe',
                 RFE(estimator=RandomForestClassifier(),
                     n_features_to_select=20))])

In [40]:
x_teste_transformed = pipe.transform(x_teste)
x_treino_transformed = pipe.transform(x_treino_res)
x_calibracao_transformed = pipe.transform(x_calibracao)
print('Shape dos dados de teste:', x_teste_transformed.shape)
print('Shape dos dados de treino:', x_treino_transformed.shape)
print('Shape dos dados de calibracao', x_calibracao_transformed.shape)

Shape dos dados de teste: (7946, 20)
Shape dos dados de treino: (117218, 20)
Shape dos dados de calibracao (7945, 20)


# Salvando os dados

In [43]:
dados_tratado ={}
dados_tratado["x_treino"] = x_treino_transformed
dados_tratado["y_treino"] = y_treino_res.values
dados_tratado["x_teste"] = x_teste_transformed
dados_tratado["y_teste"] = y_teste.values
dados_tratado['x_calibracao'] = x_calibracao_transformed
dados_tratado['y_calibracao'] = y_calibracao.values
with open("dados/dados_tratados.pickle", "wb") as f:
    pickle.dump(dados_tratado, f)